In [2]:
import os
import csv
from datasets import load_dataset
from huggingface_hub import notebook_login

notebook_login()

In [3]:

CAD_PATH = "/media/chunt/6F53-5578/CAD_img/"
TZVET_PATH = "/media/chunt/6F53-5578/TZVET_img/"

CAD_dir = [(CAD_PATH+f,f) for f in os.listdir(CAD_PATH) if os.path.isdir(os.path.join(CAD_PATH,f))]
#TZVET_dir = [(TZVET_PATH+f,f) for f in os.listdir(TZVET_PATH) if os.path.isdir(os.path.join(TZVET_PATH,f))]

for d in CAD_dir[:4]:
    img_dir = d[0]
    label = d[1] if len(d[1]) < 40 else d[1][:40].rstrip("-")
    artist = label.split("-at-")[0].rstrip("-")
    data = [["file_name", "text"]]
    count = 0
    for filename in os.listdir(img_dir):
        if filename.endswith(".jpg"):
            src = os.path.join(img_dir, filename)
            dst = os.path.join(img_dir, f"img{count}.jpg")
            os.rename(src, dst)
            data.append([f"img{count}.jpg", artist])
            count += 1
        if filename.endswith(".pdf"):
            os.remove(os.path.join(img_dir, filename))

    metadata_filename = img_dir + "/metadata.csv"
    with open(metadata_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        for row in data:
            writer.writerow(row)
    repo_name = label if len(label) < 96 else label[:96]
    print(artist)
    #dataset = load_dataset("imagefolder", data_dir=img_dir, drop_labels=True)
    #dataset.push_to_hub(f"Chunt0/{repo_name}")




seth-price
daniel-graham-loxton
karol-radziszewski
kerstin-cmelka


In [5]:
#notebook_login()
dataset = load_dataset("Chunt0/karol-radziszewski-at-between-bridges-be")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/36 [00:00<?, ? examples/s]